In [1]:
import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

import numpy as np
import pandas as pd
import json
import os

In [13]:
potential = 'DW'
cutoff = 16
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\Differential Evolution\PauliDecomp\VQE\AnsatzSteps\{}\{}{}".format(potential,potential,cutoff))


dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])

    success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [14]:
df = pd.DataFrame({
    'folder': folders,
    #'num_vqe': num_vqe,
    'shots': shots,
    'tolerance': tolerance,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [ ]:
df[['Delta min E', 'Delta median E']] = df[['Delta min E', 'Delta median E']].astype(float)
df = df.sort_values('Delta median E', ascending=False)
df.to_excel(os.path.join(folder_path, "table.xlsx"), index=False)
df

,folder,shots,tolerance,Converged Runs,Mean Iter,Delta min E,Delta median E,Exact,Time
3,RY_CRY_3RY,10000,0.1,100/100,1020,0.908266,0.908264,8.91599e-01,"2 days, 19:28:05.345281"
5,RY_CRY_3RY_CRY,10000,0.1,100/100,2348,0.908266,0.908264,8.91599e-01,"7 days, 15:07:19.130052"
4,RY_CRY_3RY_2CRY,10000,0.1,87/100,5040,0.908266,0.908263,8.91599e-01,"19 days, 5:38:25.540263"
2,RY_CRY_2RY,10000,0.1,100/100,669,0.908266,0.908261,8.91599e-01,"1 day, 11:49:48.781278"
6,RY_CRY_RY,10000,0.1,100/100,441,0.908266,0.908258,8.91599e-01,18:27:17.000264
1,RY_CRY,10000,0.1,100/100,375,0.908266,0.908238,8.91599e-01,9:52:36.743701
0,RY,10000,0.1,100/100,206,0.908266,0.908112,8.91599e-01,2:47:26.774585


: 